In [16]:
# from gradio_client import Client

# client = Client("https://mbazanlp-finetuned-nllb-general-en-kin.hf.space/")
# result = client.predict(
# 				"Howdy!",	
# 				"eng_Latn",	
								
# 				"kin_Latn",	
							
# 				api_name="/predict"
# )
# print(result)

In [17]:
import re

def fileToSent(filename):
    sentences = []
    with open(filename, 'r') as file:
        line_count = 0
        for line in file:
            line_sentences = re.split(r'(?<=[.!?])\s+', line.strip())
            for sent in line_sentences:
                if len(sent) > 10 and len(sent.split()) > 4 and len(sent.split()) < 10 and sent[0].isupper():
                    # Remove non-alphanumeric characters
                    sent = re.sub(r'[^a-zA-Z0-9\s]', '', sent)
                    # Remove extra white spaces within the sentence
                    sent = re.sub(r'\s+', ' ', sent).strip()
                    # Remove leading numbers until the sentence begins with a letter
                    sent = re.sub(r'^\d+\s', '', sent)
                    sentences.append(sent)
    return sentences

In [18]:
mag_sentences = fileToSent('coca/text_mag.txt')
fic_sentences = fileToSent('coca/text_fic.txt')
news_sentences = fileToSent('coca/text_news.txt')
spok_sentences = fileToSent('coca/text_spok.txt')
acad_sentences = fileToSent('coca/text_acad.txt')
web_sentences = fileToSent('coca/text_web.txt')
tvm_sentences = fileToSent('coca/text_tvm.txt')
# concat all sentences
sentences = mag_sentences + fic_sentences + news_sentences + spok_sentences + acad_sentences + web_sentences + tvm_sentences

In [19]:
print(fic_sentences[:5])

['Of a heroin overdose in a cheap motel', 'He died before he got old', 'Paine died in 1976 of a drug overdose', 'You know dateless wonders', 'I always preferred the big monster flicks']


In [20]:
# EngToKin (EnglishText) -> KinText
def EngToKin(EnglishText, client):
    result = client.predict(
                    EnglishText,	
                    "eng_Latn",	
                    "kin_Latn",
                    api_name="/predict"
    )
    return result


In [21]:
from deep_translator import GoogleTranslator
def BatchEngToKin(batch):
    my_translator = GoogleTranslator(source='en', target='rw')
    result = my_translator.translate_batch(batch)
    return result

In [30]:
translated = BatchEngToKin(spok_sentences[9000:11000])


In [28]:
print(translated)

['Byitwa ubujura', 'Kandi ibyo twabyita ubusosiyalisiti', 'Hano haribyo', 'Ni ikarito rwose yari ifite uko yakabaye', 'Inyuguti za Cartoon zivanze nibiganiro nyabyo', 'Ibi ntabwo ntabikora nubwo', 'Ntibisanzwe ariko byumvikana', 'Kandi uzi ko byari a', 'Ntabwo njya muri ibyo', 'Wabonye ABC NBC', 'Ibyo birengagiza leta s', 'Ndashaka kuvuga ko aribyo', 'Nzi ko ubishaka', 'Ndi umurwa mukuru', 'Ntabwo ndi umudendezo wibicucu', 'Wavuze ko wongeye kuba capitaliste', 'Muraho reka twibagirwe amateka OK', 'Ntushobora kwibagirwa amateka', 'San Diego Kansas Umujyi wa Texas', 'Bitera inyungu', 'Irangira ite', 'Turashaka kuvuga rwose', 'Tuzasubira inyuma', 'Murakoze mwembi kuba twifatanije natwe', 'Steve reka ntangire nawe', 'Byarambabaje cyane', "Ndetse n'ibi", 'Oya hano s ikintu', 'Kandi ikindi kintu', 'Dukeneye abantu benshi bakize muri iki gihugu', 'Tugomba gutunga abantu', 'Ni uruhe rwitwazo kuri ibyo', 'Nibyo kopi', 'Ntekereza ko bitandukanye cyane', 'Ibigoye ntabwo ari amakosa kuri Wall Stre

In [24]:
import nltk
from nltk.translate import bleu
#calculateBLEU(hypothesis, reference) -> BLEUscore (float between 0 and 1)
def calculateBLEU(hypothesis, reference):
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
    return BLEUscore

ModuleNotFoundError: No module named 'xml.etree'

In [ ]:
def DictBLEU(Sentences, testCol, outputCol):
    for i in Sentences:
        BLEUscore = calculateBLEU(Sentences[i][1], Sentences[i][testCol])
        Sentences[i][outputCol] = BLEUscore
    return Sentences

In [ ]:
DictBLEU(Sentences, 2, 5)
DictBLEU(Sentences, 3, 6)
DictBLEU(Sentences, 4, 7)

In [29]:
# Write the updated data to a new CSV file
import csv


with open('trans.csv', 'a', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    for i in range(len(translated)):
        csv_writer.writerow([translated[i], spok_sentences[i+7000]])
        